<div class="alert alert-warning">
    
<b>Disclaimer:</b> 
    
The main objective of the <i>Jupyter</i> notebooks is to show how to use the models of the <i>QENS library</i> by
    
- building a fitting model: composition of models, convolution with a resolution function  
- setting and running the fit  
- extracting and displaying information about the results  

These steps have a minimizer-dependent syntax. That's one of the reasons why different minimizers have been used in the notebooks provided as examples.  
But, the initial guessed parameters might not be optimal, resulting in a poor fit of the reference data.

</div>

# Gaussian Model 3D with lmfit

## Introduction

<div class="alert alert-info">
    
The objective of this notebook is to show how to use the <b>Gaussian Model 3D</b> model to perform some 
fits using <a href="https://lmfit.github.io/lmfit-py/">lmfit</a>.
</div>

### Physical units
For information about unit conversion, please refer to the jupyter notebook called `Convert_units.ipynb` in the `tools` folder.

The dictionary of units defined in the cell below specify the units of the refined parameters adapted to the convention used in the experimental datafile.

In [ ]:
# Units of parameters for selected QENS model and experimental data
dict_physical_units = {'omega': "meV", 
                       'q': "1/Angstrom", 
                       'D': "meV.Angstrom^2", 
                       'variance_ux': "Angstrom", 
                       'scale': "unit_of_signal.meV",
                       'center': "meV"}

## Importing the required librairies

In [ ]:
# import python modules for plotting, fitting
from __future__ import print_function
import numpy as np

import matplotlib.pyplot as plt

# for interactivity (plots, buttons...)
import ipywidgets

from lmfit import Model, Parameters

import QENSmodels

## Plot of the fitting model

The widget below shows the peak shape function imported from QENSmodels where the function's parameters can be varied.

In [ ]:
# Dictionary of initial values
ini_parameters = {'q': 1., 'scale': 5., 'center': 0., 'D': 1., 'variance_ux': 1.}

def interactive_fct(q, scale, center, D, variance_ux):
    xs = np.linspace(-10, 10, 100)
    
    fig1, ax1 = plt.subplots()
    ax1.plot(xs, QENSmodels.sqwGaussianModel3D(xs, q, scale, center, D, variance_ux))
    ax1.set_xlabel('x')
    ax1.grid()

# Define sliders for modifiable parameters and their range of variations

q_slider = ipywidgets.FloatSlider(value=ini_parameters['q'],
                                  min=0.1, max=10., step=0.1,
                                  description='q', 
                                  continuous_update=False) 

scale_slider = ipywidgets.FloatSlider(value=ini_parameters['scale'],
                                      min=0.1, max=10, step=0.1,
                                      description='scale',
                                      continuous_update=False) 

center_slider = ipywidgets.IntSlider(value=ini_parameters['center'],
                                     min=-10, max=10, step=1,
                                     description='center', 
                                     continuous_update=False) 

D_slider = ipywidgets.FloatSlider(value=ini_parameters['D'],
                                     min=0.1, max=10, step=0.1,
                                     description='D',
                                     continuous_update=False)

variance_ux_slider = ipywidgets.FloatSlider(value=ini_parameters['variance_ux'],
                                       min=0.1, max=10, step=0.1,
                                       description='variance_ux',
                                       continuous_update=False)

grid_sliders = ipywidgets.HBox([ipywidgets.VBox([q_slider, scale_slider, center_slider])
                                ,ipywidgets.VBox([D_slider, variance_ux_slider])])

# Define function to reset all parameters' values to the initial ones
def reset_values(b):
    """Reset the interactive plots to inital values."""
    q_slider.value = ini_parameters['q'] 
    scale_slider.value = ini_parameters['scale'] 
    center_slider.value = ini_parameters['center']  
    D_slider.value = ini_parameters['D'] 
    variance_ux_slider.value = ini_parameters['variance_ux'] 

# Define reset button and occurring action when clicking on it
reset_button = ipywidgets.Button(description = "Reset")
reset_button.on_click(reset_values)

# Display the interactive plot
interactive_plot = ipywidgets.interactive_output(interactive_fct,       
                                         {'q': q_slider, 
                                          'scale': scale_slider,
                                          'center': center_slider,
                                          'D': D_slider,
                                          'variance_ux': variance_ux_slider})  
                                            
display(grid_sliders, interactive_plot, reset_button)

## Creating the reference data

In [ ]:
nb_points = 200
xx = np.linspace(-2, 2, nb_points)
added_noise = np.random.normal(0, 1, nb_points)

gaussian_model_3d_noisy = QENSmodels.sqwGaussianModel3D(
    xx,
    q=.5,
    scale=0.7,
    center=0.3,
    D=1,
    variance_ux=1.2
) * (1 + 0.1 * added_noise) + 0.01 * added_noise

# plot initial mode
fig0, ax0 = plt.subplots()
ax0.plot(xx, gaussian_model_3d_noisy)
ax0.grid();

## Setting and fitting

In [ ]:
gmodel = Model(QENSmodels.sqwGaussianModel3D)

print('Names of parameters:', gmodel.param_names)
print('Independent variable(s):', gmodel.independent_vars)

initial_parameters_values = [1.22, 0.25, .3, 0.33]

# Define boundaries for parameters to be refined
gmodel.set_param_hint('scale', min=0)
gmodel.set_param_hint('center', min=-5, max=5)
gmodel.set_param_hint('D', min=0)
gmodel.set_param_hint('variance_ux', min=0)

# Fix some of the parameters
gmodel.set_param_hint('q', vary=False)

# Fit
result = gmodel.fit(gaussian_model_3d_noisy, w=xx, q=1.,
                    scale=initial_parameters_values[0], 
                    center=initial_parameters_values[1],
                    D=initial_parameters_values[2],
                    variance_ux=initial_parameters_values[3])

In [ ]:
# Plot Initial model and reference data
fig1, ax1 = plt.subplots()
ax1.plot(xx, gaussian_model_3d_noisy, 'b-', label='reference data')
ax1.plot(xx, result.init_fit, 'k--', label='model with initial guesses')
ax1.set(xlabel='x', title='Initial model and reference data')
ax1.grid()
ax1.legend();

## Plotting results

using methods implemented in `lmfit`

In [ ]:
# display result
print('Result of fit:\n',result.fit_report())

# plot fitting results using lmfit functionality
result.plot();

In [ ]:
# plot fitting results and reference data using matplotlib.pyplot
fig2, ax2 = plt.subplots()
ax2.plot(xx, gaussian_model_3d_noisy, 'b-', label='reference data')
ax2.plot(xx, result.best_fit, 'r.', label='fitting result')
ax2.legend()
ax2.set(xlabel='x', title='Fit result and reference data')
ax2.grid();